# Imports

In [17]:
import pandas as pd
import numpy as np
from sklearn import neighbors
from scipy.sparse import coo_matrix
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [18]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


# Defining constants

- `NUMBER_OF_NEIGHBOURS` specifies the number of neighbors to consider in the KNN algorithm.
- `RECOMMENDATIONS_LIMIT` limits the number of recommendations to be provided.
- `MINIMAL_NUMBER_USER_REVIEWS` set the thresholds for the minimum number of reviews required for users.
- `MINIMAL_NUMBER_BOOK_REVIEWS` set the thresholds for the minimum number of reviews required for books.

In [19]:
NUMBER_OF_NEIGHBOURS = 5
RECOMMENDATIONS_LIMIT = 5
MINIMAL_NUMBER_USER_REVIEWS = 100
MINIMAL_NUMBER_BOOK_REVIEWS = 50
METRIC = 'cosine'

# Data loading


We load the data from CSV files into pandas DataFrames. The datasets are sourced from the [Book Recommendation Dataset on Kaggle](https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset/data). Below is a brief description of each dataset, including its size and the columns it contains.

1. **Books Dataset (`df_books`)**:
   - **Size**: 271,360 rows, 8 columns
   - **Columns**: 
     - `ISBN`: Unique identifier for books
     - `Book-Title`: Title of the book
     - `Book-Author`: Author of the book
     - `Year-Of-Publication`: Year the book was published
     - `Publisher`: Publisher of the book
     - `Image-URL-S`: URL of the small image of the book cover
     - `Image-URL-M`: URL of the medium image of the book cover
     - `Image-URL-L`: URL of the large image of the book cover

2. **Users Dataset (`df_users`)**:
   - **Size**: 278,858 rows, 3 columns
   - **Columns**: 
     - `User-ID`: Unique identifier for users
     - `Location`: Location of the user
     - `Age`: Age of the user

3. **Ratings Dataset (`df_ratings`)**:
   - **Size**: 1,149,780 rows, 3 columns
   - **Columns**: 
     - `User-ID`: Unique identifier for users
     - `ISBN`: Unique identifier for books
     - `Book-Rating`: Rating given by the user to the book

In [20]:
df_books = pd.read_csv('data/books.csv')
df_users = pd.read_csv('data/users.csv')
df_ratings = pd.read_csv('data/ratings.csv')

C:\Users\Konrad\AppData\Local\Temp\ipykernel_27884\551020402.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_books = pd.read_csv('data/books.csv')


# Data processing

This section processes the loaded data. Duplicates in the book titles are removed. We count the number of reviews for each book and each user, creating new DataFrames `df_user_reviews_count` and `df_book_reviews_count`. These are merged with `df_users` and `df_books` to include the number of reviews and calculate the weighted average rating for each book. These are temporary variables that will be used later.

In [21]:
display(df_books[df_books['Book-Title'] == "Harry Potter and the Goblet of Fire (Book 4)"])
df_books = df_books.drop_duplicates(subset=['Book-Title'], keep='first')
display(df_books[df_books['Book-Title'] == "Harry Potter and the Goblet of Fire (Book 4)"])

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
5431,0439139597,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,2000,Scholastic,http://images.amazon.com/images/P/0439139597.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0439139597.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0439139597.01.LZZZZZZZ.jpg
6932,0439139600,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,2002,Scholastic Paperbacks,http://images.amazon.com/images/P/0439139600.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0439139600.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0439139600.01.LZZZZZZZ.jpg


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
5431,0439139597,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,2000,Scholastic,http://images.amazon.com/images/P/0439139597.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0439139597.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0439139597.01.LZZZZZZZ.jpg


In [22]:
books_by_number_of_reviews = df_ratings['ISBN'].value_counts()
users_by_number_of_reviews = df_ratings['User-ID'].value_counts()

df_user_reviews_count = users_by_number_of_reviews.reset_index()
df_user_reviews_count.columns = ['User-ID', 'Number_of_Reviews']
df_users_with_reviews = df_users.merge(df_user_reviews_count, on='User-ID', how='left')
df_users_with_reviews['Number_of_Reviews'].fillna(0, inplace=True)

df_book_reviews_count = books_by_number_of_reviews.reset_index()
df_book_reviews_count.columns = ['ISBN', 'Number_of_Reviews']
df_books_with_reviews = df_books.merge(df_book_reviews_count, on='ISBN', how='left')
df_books_with_reviews['Number_of_Reviews'].fillna(0, inplace=True)

weighted_average = df_ratings.groupby('ISBN')['Book-Rating'].mean()
df_books_with_reviews = df_books_with_reviews.merge(weighted_average, on='ISBN', how='left')
df_books_with_reviews.rename(columns={'Book-Rating': 'Weighted_Average_Rating'}, inplace=True)

We update the `df_books` DataFrame by merging it with `df_books_with_reviews` to include the weighted average rating. Columns that are not needed are dropped. Similarly, we update the `df_users` DataFrame by dropping unnecessary columns.

In [23]:
df_books = df_books.merge(df_books_with_reviews[['ISBN', 'Weighted_Average_Rating']], on='ISBN', how='left')
df_books = df_books.drop(columns=['Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L'])

df_users = df_users.drop(columns=['Location', 'Age'])

We filter the ratings data to include only users and books that meet the minimum review thresholds. We also ensure that only books present in `df_books` are included.

In [24]:
print(f"Before filtering {len(df_ratings)}")

df_drop_books = df_books_with_reviews[df_books_with_reviews['Number_of_Reviews'] < MINIMAL_NUMBER_BOOK_REVIEWS]
df_drop_users = df_users_with_reviews[df_users_with_reviews['Number_of_Reviews'] < MINIMAL_NUMBER_USER_REVIEWS]

df_ratings = df_ratings[df_ratings['ISBN'].isin(df_books['ISBN'])]

df_ratings = df_ratings[  ~df_ratings['ISBN'].isin(df_drop_books['ISBN'])]
df_ratings = df_ratings[~df_ratings['User-ID'].isin(df_drop_users['User-ID'])]

print(f"Before filtering {len(df_ratings)}")

Before filtering 1149780
Before filtering 98313


# Creating the sparse matrix

We create a sparse matrix of ratings using the `coo_matrix` function from `scipy.sparse`. A sparse matrix is a type of matrix that is optimized for storing data where most of the elements are zero. This is particularly useful in the context of recommendation systems, where the matrix representing user ratings for items is typically very sparse – meaning most users have rated only a small subset of items.

In this case, the sparse matrix `ratings_matrix` stores the ratings given by users to books. Each element in the matrix corresponds to a rating, with rows representing users and columns representing books. Using a sparse matrix helps to save memory and computational resources when working with large datasets.

For example, let's consider a simple scenario with three users and four books. The ratings matrix might look like this:

\begin{bmatrix}
0 & 3 & 0 & 1 \\
4 & 0 & 0 & 0 \\
0 & 0 & 5 & 2 \\
\end{bmatrix}

Here, user 1 has rated books 2 and 4, user 2 has rated book 1, and user 3 has rated books 3 and 4. The sparse matrix representation of this data will only store the non-zero elements along with their row and column indices, which significantly reduces the amount of space needed.

The `coo_matrix` function takes three arguments: the data (ratings), row indices, and column indices. This allows us to efficiently construct the sparse matrix from the `df_ratings` DataFrame.

In [25]:
df_ratings['User_Index'] = df_ratings['User-ID'].astype("category").cat.codes
df_ratings['Book_Index'] = df_ratings['ISBN'].astype("category").cat.codes

ratings_matrix_coo = coo_matrix((df_ratings['Book-Rating'], (df_ratings['User_Index'], df_ratings['Book_Index'])))
ratings_matrix = ratings_matrix_coo.tocsr()

In [26]:
pd.DataFrame.sparse.from_spmatrix(ratings_matrix,
                                  index=df_ratings['User-ID'].astype("category").cat.categories,
                                  columns=df_ratings['ISBN'].astype("category").cat.categories)

000649840X  0007110928  002026478X  0020442203  002542730X  \
254              0           0           0           0           0   
507              0           0           0           0           0   
882              0           0           0           0           0   
1424             0           0           0           0           0   
1435             0           0           0           0           0   
...            ...         ...         ...         ...         ...   
277478           0           0           0           0           0   
277639           0           0           0           0           0   
278137           0           0           0           0           0   
278188           0           0           0           0           0   
278418           0           0           0           0           0   

        0028604199  006000438X  0060008776  006001203X  0060083948  \
254              0           0           0           0           0   
507              0           0           0           0           0   
882              0           0           0           0           0   
1424             0           0           0           0           0   
1435             0           0           0           0           0   
...            ...         ...         ...         ...         ...   
277478           0           0           0           0           0   
277639           0           0           0           0           0   
278137           0           0           0           0           0   
278188           0           0           0           0           0   
278418           0           0           0           0           0   

        0060085444  0060096195  0060168013  006016848X  0060173289  \
254              0           0           0           0           0   
507              0           0           0           0           0   
882              0           0           0           0           0   
1424             0           0           0           0           0   
1435             0           0           0           0           0   
...            ...         ...         ...         ...         ...   
277478           0           0           0           0           0   
277639           0           0           0           0           0   
278137           0           0           0           0           0   
278188           0           0           0           0           0   
278418           0           0           0           0           0   

        0060175400  0060175966  0060188731  0060192119  006019491X  \
254              0           0           0           0           0   
507              0           0           0           0           0   
882              0           0           0           0           0   
1424             0           0           0           0           0   
1435             0           0           0           0           0   
...            ...         ...         ...         ...         ...   
277478           0           0           0           0           0   
277639           0           0           0           0           0   
278137           0           0           0           0           0   
278188           0           0           0           0           0   
278418           0           0           0           0           0   

        0060199652  0060391626  0060392452  0060502258  0060505885  \
254              0           0           0           0           0   
507              0           0           0           0           0   
882              0           0           0           0           0   
1424             0           0           0           0           0   
1435             0           0           0           0           0   
...            ...         ...         ...         ...         ...   
277478           0           0           0           0           0   
277639           0           0           0           0           0   
278137        

# Model

We create and train a K-Nearest Neighbors model using the cosine similarity metric. The model is trained on the sparse matrix of ratings.

In [27]:
train_data, test_data = train_test_split(df_ratings, test_size=0.20, random_state=42)
user_index_mapping = {user_id: index for index, user_id in enumerate(train_data['User-ID'].unique())}
book_index_mapping = {isbn: index for index, isbn in enumerate(train_data['ISBN'].unique())}
train_data['User-ID'] = train_data['User-ID'].map(user_index_mapping)
train_data['ISBN'] = train_data['ISBN'].map(book_index_mapping)

train_ratings_matrix_coo = coo_matrix((train_data['Book-Rating'], (train_data['User-ID'], train_data['ISBN'])))
train_ratings_matrix = train_ratings_matrix_coo.tocsr()

In [28]:
pipeline = Pipeline([
  ('scaler', StandardScaler(with_mean=False)),
  ('knn', neighbors.NearestNeighbors(n_neighbors=NUMBER_OF_NEIGHBOURS))
])

In [29]:
pipeline.fit(train_ratings_matrix)
model = pipeline.named_steps['knn']

This function predicts the rating for a given user and book using a KNN model. It finds the nearest neighbors of the user, calculates a weighted sum of their ratings for the book, and returns the predicted rating. If no similar users have rated the book, it returns the average rating from the training data.

In [30]:
def get_predicted_rating(model, train_ratings_matrix, user_index_mapping, book_index_mapping, user_id, isbn):
  try:
    user_index = user_index_mapping[user_id]
    book_index = book_index_mapping[isbn]
  except KeyError:
    return np.nan

  distances, indices = model.kneighbors(train_ratings_matrix[user_index], n_neighbors=model.n_neighbors)
  similar_users = indices.flatten()[1:]
  distances = distances.flatten()[1:]

  weighted_sum = 0
  sum_of_weights = 0

  for i, similar_user in enumerate(similar_users):
    weight = 1 / (distances[i] + 1e-9)
    rating = train_ratings_matrix[similar_user, book_index]
    if rating > 0:
      weighted_sum += weight * rating
      sum_of_weights += weight

  if sum_of_weights == 0:
    return np.mean(train_data['Book-Rating'])

  return weighted_sum / sum_of_weights

This function calculates the **RMSE** (Root Mean Squared Error) for the model's predictions on the test data. It applies the `get_predicted_rating` function to each row in the test set to get predicted ratings, calculates the mean squared error between the actual and predicted ratings, and returns the square root of this value.

In [31]:
def calculate_rmse(model, test_data, user_index_mapping, book_index_mapping):
  test_data['Predicted_Rating'] = test_data.apply(
      lambda row: get_predicted_rating(model, train_ratings_matrix, user_index_mapping, book_index_mapping, row['User-ID'], row['ISBN']), axis=1)
  test_data = test_data.dropna(subset=['Predicted_Rating'])
  mse = mean_squared_error(test_data['Book-Rating'], test_data['Predicted_Rating'])
  rmse = np.sqrt(mse)
  return rmse

This function optimizes the KNN model by trying different combinations of the number of neighbors and distance metrics. It calculates the RMSE for each combination and returns the model, parameters, and **RMSE** that give the best performance.

In [32]:
def optimize_knn(test_data, user_index_mapping, book_index_mapping):
  param_grid = {
    'knn__n_neighbors': [5, 10, 15, 20],
    'knn__metric': ['cosine', 'euclidean', 'manhattan']
  }

  best_rmse = float('inf')
  best_params = None
  best_model = None

  for params in ParameterGrid(param_grid):
    pipeline.set_params(**params)
    pipeline.fit(train_ratings_matrix)
    model = pipeline.named_steps['knn']
    rmse = calculate_rmse(model, test_data, user_index_mapping, book_index_mapping)
    print(f"{params['knn__metric']} - {params['knn__n_neighbors']} - {rmse}")
    if rmse < best_rmse:
      best_rmse = rmse
      best_params = params
      best_model = model

  return best_model, best_params, best_rmse


In [31]:
best_model, best_params, best_rmse = optimize_knn(test_data, user_index_mapping, book_index_mapping)
print(f'Best RMSE: {best_rmse}')
print(f'Best Parameters: {best_params}')

#### It takes quite a long so here are the results:

| Metric    | Number of neighbors | RMSE              |
|-----------|---------------------|-------------------|
| Cosine    | 5                   | 3.7014430490726746|
| Cosine    | 10                  | 3.859066376211482 |
| Cosine    | 15                  | 4.0459042457010215|
| Cosine    | 20                  | 4.230951176680625 |
| Euclidean | 5                   | 3.618241038045172 |
| Euclidean | 10                  | 3.6185760810664753|
| Euclidean | 15                  | 3.6215832154460563|
| Euclidean | 20                  | 3.621126836610857 |
| Manhattan | 5                   | 3.621065160113916 |
| Manhattan | 10                  | 3.6213668960728786|
| Manhattan | 15                  | 3.6216534847469752|
| Manhattan | 20                  | 3.6209454011095135|


This function takes a user ID as input and returns a list of recommended books for that user. The recommendation process involves the following steps:

1. **Finding the nearest neighbors**:
   The function identifies the nearest neighbors of the user using the KNN model. It uses the `kneighbors` method of the model to find similar users based on their rating patterns. The number of neighbors considered is specified by the `NUMBER_OF_NEIGHBOURS` constant. We are skipping the head of the list, because it is a user we have provided.

2. **Collecting ratings from similar users**:
   For each similar user, the function collects the ratings they have given to books. It specifically looks for books that the target user has not rated yet. These ratings are stored in the `similar_users_ratings` list, which contains tuples of the book's ISBN and the rating given by the similar user.

3. **Calculating the mean rating**:
   The mean rating for each book is calculated from the collected ratings. This mean rating represents the average of the ratings given by similar users to a particular book. It is stored in the `Mean_Rating` column of the `book_recs` DataFrame.

4. **Calculating the adjusted count of ratings**:
   The adjusted count of ratings for each book is calculated to account for the popularity of the book among similar users. It is derived by multiplying the raw count of ratings (`Count_Rating`) by the ratio of the raw count to the total number of ratings from similar users. This adjustment ensures that books with a higher number of ratings have a more significant impact.

   $$
   \text{Adjusted Count} = \text{Count_Rating} \times \left( \frac{\text{Count_Rating}}{\sum \text{Count_Rating}} \right)
   $$

5. **Calculating the score**:
   The score for each book is calculated by multiplying the mean rating by the adjusted count of ratings. This score reflects both the average rating and the relative popularity of the book among similar users. Books with higher scores are considered better recommendations.

   $$
   \text{Score} = \text{Mean Rating} \times \text{Adjusted Count}
   $$

6. **Sorting the recommendations**:
   The books are sorted by their calculated scores in descending order. The number of top recommendations is selected based on the `RECOMMENDATIONS_LIMIT` constant.

In [33]:
pipeline = Pipeline([
  ('scaler', StandardScaler(with_mean=False)),
  ('knn', neighbors.NearestNeighbors(n_neighbors=NUMBER_OF_NEIGHBOURS, metric=METRIC))
])
pipeline.fit(ratings_matrix)
model = pipeline.named_steps['knn']

In [34]:
def get_recommends(user_id):
  try:
    user_index = df_ratings[df_ratings['User-ID'] == user_id]['User_Index'].values[0]
  except KeyError:
    print(f'The given user ID {user_id} does not fulfill requested number of reviews and was removed from database')
    return

  distances, indices = model.kneighbors(ratings_matrix[user_index], n_neighbors=NUMBER_OF_NEIGHBOURS)
  similar_users = indices.flatten()[1:]
  
  similar_users_ratings = []

  for i, similar_user in enumerate(similar_users):
    books_read_by_similar_user = ratings_matrix[similar_user].toarray().flatten()
    user_ratings = ratings_matrix[user_index].toarray().flatten()

    for book_index, rating in enumerate(books_read_by_similar_user):
      if rating > 0 and user_ratings[book_index] == 0:
        isbn = df_ratings[df_ratings['Book_Index'] == book_index]['ISBN'].values[0]
        similar_users_ratings.append((isbn, rating))

  book_recs = pd.DataFrame(similar_users_ratings, columns=['ISBN', 'Rating'])
  book_recs = book_recs.groupby('ISBN').agg(['mean', 'count']).reset_index()
  book_recs.columns = ['ISBN', 'Mean_Rating', 'Count_Rating']

  book_recs['Adjusted_Count'] = book_recs['Count_Rating'] * (book_recs['Count_Rating'] / book_recs['Count_Rating'].sum())
  book_recs['Score'] = book_recs['Mean_Rating'] * book_recs['Adjusted_Count']

  book_recs = book_recs.sort_values('Score', ascending=False)

  recommendations = []
  for _, row in book_recs.iterrows():
    isbn = row['ISBN']
    book_details = df_books[df_books['ISBN'] == isbn].iloc[0]
    recommendations.append((book_details['Book-Title'], book_details['Book-Author'], row['Score']))

  return recommendations[:RECOMMENDATIONS_LIMIT]

This function takes a user ID as input and returns a list of books that the user has rated along with the user's rating and the weighted average rating of each book.

In [35]:
def get_user_ratings(user_id):
  user_ratings = df_ratings[df_ratings['User-ID'] == user_id]
  rated_books = []
  
  for _, row in user_ratings.iterrows():
    isbn = row['ISBN'].strip().lower()
    book_details = df_books[df_books['ISBN'].str.strip().str.lower() == isbn].iloc[0]
    rated_books.append(
        (book_details['Book-Title'], book_details['Book-Author'], row['Book-Rating'], book_details['Weighted_Average_Rating']))

  return rated_books

### Example

Let's assume that User A has rated the books as follows:
- Book 1: no rating
- Book 2: no rating
- Book 3: rating 4
- Book 4: rating 5

Similar users have rated the books as follows:
- User B: Book 1: 3, Book 2: 4, Book 3: no rating, Book 4: 5
- User C: Book 1: 4, Book 2: 3, Book 3: no rating, Book 4: 4
- User D: Book 1: no rating, Book 2: 5, Book 3: no rating, Book 4: 4

The steps for the calculations are as follows:

1. **Mean Rating**:
   - Book 1: (3 + 4) / 2 = 3.5
   - Book 2: (4 + 3 + 5) / 3 = 4.0

2. **Adjusted Count of Ratings**:
   - Total count of ratings: 2 (Book 1) + 3 (Book 2) = 5
   - Book 1: 2 * (2 / 5) = 0.8
   - Book 2: 3 * (3 / 5) = 1.8

3. **Score**:
   - Book 1: 3.5 * 0.8 = 2.8
   - Book 2: 4.0 * 1.8 = 7.2

Based on the scores, the books are sorted, and the top recommendations are:
1. Book 2: Score 7.2
2. Book 1: Score 2.8

In [36]:
user_id = 254

recommended_books = get_recommends(user_id)
user_rated_books = get_user_ratings(user_id)

df_rated_books = (pd.DataFrame(user_rated_books, columns=['Title', 'Author', 'Rating', 'Weighted average rating'])
                  .sort_values(by=['Rating'], ascending=False))
df_recommended_books = pd.DataFrame(recommended_books, columns=['Title', 'Book author', 'Score'])

In [37]:
df_rated_books

,Title,Author,Rating,Weighted average rating
17,American Gods,Neil Gaiman,10,4.006623
18,American Gods: A Novel,Neil Gaiman,9,3.307692
45,1984,George Orwell,9,4.614583
25,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,9,4.729345
26,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,9,6.467005
...,...,...,...,...
32,The Outsiders,S. E. Hinton,0,3.702703
1,The Poisonwood Bible: A Novel,Barbara Kingsolver,0,3.609312
34,"Dune Messiah (Dune Chronicles, Book 2)",Frank Herbert,0,3.145161
35,Neuromancer (Remembering Tomorrow),William Gibson,0,3.273743


In [38]:
df_recommended_books

,Title,Book author,Score
0,Harry Potter and the Order of the Phoenix (Book 5),J. K. Rowling,4.444444
1,Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback)),J. K. Rowling,1.111111
2,Don't Sweat the Small Stuff and It's All Small Stuff : Simple Ways to Keep the Little Things from Taking Over Your Life (Don't Sweat the Small Stuff Series),Richard Carlson,1.111111
3,Chicken Soup for the Soul (Chicken Soup for the Soul),Jack Canfield,1.111111
4,Life of Pi,Yann Martel,1.000000
